In [ ]:
import warnings
warnings.simplefilter("ignore")
import gzip
import json
import pandas as pd

def parse_json_gz(file_path):
        with open(file_path, 'r') as f:
            for line in f:
                yield json.loads(line)

reviews = list(parse_json_gz(r'C:\MS DS\RG_Winter25\Capstone 1 DSCI 591\data\review-Pennsylvania.json\review-Pennsylvania.json'))
metadata = list(parse_json_gz(r'C:\MS DS\RG_Winter25\Capstone 1 DSCI 591\data\meta-Pennsylvania.json\meta-Pennsylvania.json'))


pd.set_option('display.max_columns', None)

reviews = pd.DataFrame(reviews)
metadata = pd.DataFrame(metadata)
# Identify all columns with dtype 'object'
object_cols = reviews.select_dtypes(include=['object']).columns

# Convert those columns to string
reviews[object_cols] = reviews[object_cols].astype(str)
reviews.drop_duplicates(inplace=True)
# Identify all columns with dtype 'object'
object_cols = metadata.select_dtypes(include=['object']).columns

# Convert those columns to string
metadata[object_cols] = metadata[object_cols].astype(str)
metadata.drop_duplicates(inplace=True)

print("Reviews DataFrame:")
print(reviews.info())
print("\nMetadata DataFrame:")
print(metadata.info())
def merge_datasets(reviews, metadata, merge_on='gmap_id'):
    return pd.merge(reviews, metadata, on=merge_on, how='left')


object_cols = metadata.select_dtypes(include=['object']).columns



merged_data_df = merge_datasets(reviews, metadata)
def handle_missing_values(df, state='PA'):
    drop_subset = ['user_id', 'rating']
    text_columns = ['text', 'description', 'resp']
    categorical_columns = ['name_x', 'name_y', 'category', 'price', 'address', 'hours', 'MISC', 'url']
    
    # Dropping rows with missing critical fields
    df = df.dropna(subset=drop_subset)

    # Filling text columns with empty strings
    for col in text_columns:
        if col in df.columns:
            df[col] = df[col].fillna("")

    # #Filling state value
    # if 'state' in df.columns:
    #     df['state'] = state
    
    # Filling categorical columns with "unknown"
    for col in categorical_columns:
        if col in df.columns:
            df[col] = df[col].fillna("unknown")
    
    return df
df = handle_missing_values(merged_data_df)
if 'time' in df.columns:
    try:
        df['time'] = pd.to_datetime(df['time'], unit='ms')
    except Exception:
        df['time'] = pd.to_datetime(df['time'], errors='coerce')
numeric_fields = ['rating','user_id', 'avg_rating', 'num_of_reviews', 'latitude', 'longitude']
for field in numeric_fields:
    if field in df.columns:
        df[field] = pd.to_numeric(df[field], errors='coerce')
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# import string
# import re

# nltk.download('punkt')  
# nltk.download('stopwords')  
# nltk.download('wordnet')    
# nltk.download('omw-1.4')

# def clean_text(text):
#     # changing text to Lowercase
#     text = text.lower()
    
#     # Removing punctuation
#     text = text.translate(str.maketrans('', '', string.punctuation))
    
#     # Removing extra whitespace
#     text = re.sub(r'\s+', ' ', text).strip()
    
#     # Removing stopwords
#     tokens = word_tokenize(text)
#     tokens = [word for word in tokens if word not in set(stopwords.words('english'))]
    
#     return ' '.join(tokens)
# df['clean_text'] = df['text'].apply(clean_text)
df.rename(columns={'name_x': 'customer_name', 'name_y': 'business_name'}, inplace=True)


Reviews DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 21712798 entries, 0 to 21944801
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  object 
 1   name     object 
 2   time     int64  
 3   rating   float64
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 1.5+ GB
None

Metadata DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 189836 entries, 0 to 190815
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              189836 non-null  object 
 1   address           189836 non-null  object 
 2   gmap_id           189836 non-null  object 
 3   description       189836 non-null  object 
 4   latitude          189836 non-null  float64
 5   longitude         189836 non-null  float64
 6   category          189836 non-null  object 
 7   avg_rating        189

In [14]:
object_cols = df.select_dtypes(include=['object']).columns

# Convert those columns to string
df[object_cols] = df[object_cols].astype(str)

columns_to_fix = df[object_cols] # Replace with actual column names
for col in columns_to_fix:
    df[col] = df[col].astype(str).str.replace("\x00", "", regex=True)

In [15]:
df.drop_duplicates(inplace=True)

In [19]:
# Drop specified columns
df_new = df.drop(columns=['category', 'hours', 'MISC', 'relative_results','pics'])

# Sort by 'time' column
df_new = df_new.sort_values(by='time', ascending=True)

# Reset index and set as 'id'
df_new = df_new.reset_index(drop=True).rename_axis('id').reset_index()

# Display first few rows
df_new.head()


,id,user_id,customer_name,time,rating,text,resp,gmap_id,business_name,address,description,latitude,longitude,avg_rating,num_of_reviews,price,state,url
0,0,1.136701e+20,Michelle OBrien,1990-12-31,4.0,Flatbreads are delicious for appetizers to sha...,None,0x89c6944bbcf4f9fb:0x77c6109d1ce368f4,Seasons 52,"Seasons 52, 160 N Gulph Rd Suite 101, King of ...",Rotating menu of seasonal American dishes alon...,40.085768,-75.394432,4.4,1618,$$,PA,https://www.google.com/maps/place//data=!4m2!3...
1,1,1.004704e+20,Howard Dinin,1990-12-31,4.0,"well-run and clearly a popular ""happening"" spo...",None,0x89c6c630798d07a5:0x7da1292822f531db,Alma de Cuba,"Alma de Cuba, 1623 Walnut St, Philadelphia, PA...",Fusion fare with Cuban influences; happy hour ...,39.950078,-75.168517,4.4,998,$$$,PA,https://www.google.com/maps/place//data=!4m2!3...
2,2,1.138597e+20,Joe Nicolas,1990-12-31,5.0,None,None,0x89c6c625b14dea8b:0xb31072102487e2fe,VALANNI,"VALANNI, 1229 Spruce St, Philadelphia, PA 1910...",Distinctive Mediterranean-Latin dishes served ...,39.946850,-75.162217,4.3,448,$$,PA,https://www.google.com/maps/place//data=!4m2!3...
3,3,1.004403e+20,Chef Marsha Kiv,1990-12-31,3.0,I have been to many of Chef Garces' restaurant...,None,0x89c6c6f86d1010ad:0x2fb936cf6717b44d,Distrito,"Distrito, 3945 Chestnut St, Philadelphia, PA 1...",Mexican bites & potent margaritas flavor this ...,39.955999,-75.202017,4.0,838,$$,PA,https://www.google.com/maps/place//data=!4m2!3...
4,4,1.178880e+20,Melody Tsai,1990-12-31,5.0,Great Food!,None,0x89c6c60e85b145bf:0xb189cdb4c35a41f3,South Philadelphia Tap Room,"South Philadelphia Tap Room, 1509 Mifflin St, ...",American beer bar featuring a lineup of drafts...,39.927216,-75.171477,4.6,746,$$,PA,https://www.google.com/maps/place//data=!4m2!3...


In [22]:
df_new.head(100)


,id,user_id,customer_name,time,rating,text,resp,gmap_id,business_name,address,description,latitude,longitude,avg_rating,num_of_reviews,price,state,url
0,0,1.136701e+20,Michelle OBrien,1990-12-31,4.0,Flatbreads are delicious for appetizers to sha...,None,0x89c6944bbcf4f9fb:0x77c6109d1ce368f4,Seasons 52,"Seasons 52, 160 N Gulph Rd Suite 101, King of ...",Rotating menu of seasonal American dishes alon...,40.085768,-75.394432,4.4,1618,$$,PA,https://www.google.com/maps/place//data=!4m2!3...
1,1,1.004704e+20,Howard Dinin,1990-12-31,4.0,"well-run and clearly a popular ""happening"" spo...",None,0x89c6c630798d07a5:0x7da1292822f531db,Alma de Cuba,"Alma de Cuba, 1623 Walnut St, Philadelphia, PA...",Fusion fare with Cuban influences; happy hour ...,39.950078,-75.168517,4.4,998,$$$,PA,https://www.google.com/maps/place//data=!4m2!3...
2,2,1.138597e+20,Joe Nicolas,1990-12-31,5.0,None,None,0x89c6c625b14dea8b:0xb31072102487e2fe,VALANNI,"VALANNI, 1229 Spruce St, Philadelphia, PA 1910...",Distinctive Mediterranean-Latin dishes served ...,39.946850,-75.162217,4.3,448,$$,PA,https://www.google.com/maps/place//data=!4m2!3...
3,3,1.004403e+20,Chef Marsha Kiv,1990-12-31,3.0,I have been to many of Chef Garces' restaurant...,None,0x89c6c6f86d1010ad:0x2fb936cf6717b44d,Distrito,"Distrito, 3945 Chestnut St, Philadelphia, PA 1...",Mexican bites & potent margaritas flavor this ...,39.955999,-75.202017,4.0,838,$$,PA,https://www.google.com/maps/place//data=!4m2!3...
4,4,1.178880e+20,Melody Tsai,1990-12-31,5.0,Great Food!,None,0x89c6c60e85b145bf:0xb189cdb4c35a41f3,South Philadelphia Tap Room,"South Philadelphia Tap Room, 1509 Mifflin St, ...",American beer bar featuring a lineup of drafts...,39.927216,-75.171477,4.6,746,$$,PA,https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,1.183500e+20,Dan F,2005-08-31,5.0,this could be the best byo in the burbs. ever...,None,0x89c6c07607056697:0x2b8e2876e3a9517e,Sola,"Sola, 614 Lancaster Ave, Bryn Mawr, PA 19010",New American seasonal cuisine served in a cozy...,40.018088,-75.312039,4.3,28,$$$,PA,https://www.google.com/maps/place//data=!4m2!3...
96,96,1.183500e+20,Dan F,2005-08-31,4.0,this neighborhood tap room really cooks. ever...,None,0x89c6c898b3eb1341:0x12cfe87713184d11,New Wave Cafe,"New Wave Cafe, 784 S 3rd St, Philadelphia, PA ...",Locals fill this casual sports bar to chat wit...,39.938080,-75.148527,4.4,435,$,PA,https://www.google.com/maps/place//data=!4m2!3...
97,97,1.051016e+20,Mac Frazier,2005-10-22,2.0,great traditional german food! the setting is ...,None,0x89c6afbf8417a279:0x3dd203c5ec7db1e3,Otto's Brauhaus,"Otto's Brauhaus, 233 Easton Rd, Horsham, PA 19044","Prime rib, steaks, seafood & German specialtie...",40.175800,-75.128520,4.0,678,$$,PA,https://www.google.com/maps/place//data=!4m2!3...
98,98,1.051016e+20,Mac Frazier,2005-10-22,2.0,"cute, themed diner, with good shakes and burge...",None,0x89c6c2f455344797:0x12987ea5eabacd5a,Nifty Fifty's,"Nifty Fifty's, 1900 MacDade Boulevard, Folsom,...","Nostalgic, '50s-style American diner with burg...",39.895706,-75.314895,4.4,3156,$,PA,https://www.google.com/maps/place//data=!4m2!3...


In [8]:
df_new.head().to_csv('./final_mergxzced_pa.csv',index=True)

In [23]:

import pandas as pd
from sqlalchemy import create_engine
from concurrent.futures import ThreadPoolExecutor

# Database connection parameters
DB_HOST = "localhost"  # e.g., "localhost"
DB_PORT = "5432"  # Default PostgreSQL port
DB_NAME = "Capstone"
DB_USER = "postgres"
DB_PASSWORD = "Google#13"
TABLE_NAME = "main"
CHUNK_SIZE = 600  # Adjust as needed

# Create a SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")



def insert_chunk(chunk, engine, table_name):
    """
    Inserts a chunk of pandas DataFrame rows into a table in the given database engine.
    """
    try:
        # Write to SQL in 'append' mode
        chunk.to_sql(table_name, engine, if_exists="append", index=False)
    except Exception as e:
        print(f"Error inserting chunk: {e}")


# Perform sequential chunk-wise insertion
try:
    total_rows = len(df_new)
    for start in range(0, total_rows, CHUNK_SIZE):
        # Slicing the DataFrame chunk
        chunk = df_new.iloc[start: start + CHUNK_SIZE]
        insert_chunk(chunk, engine, TABLE_NAME)
    print(f"Data successfully dumped to table '{TABLE_NAME}' in PostgreSQL!")
except Exception as e:
    print("Error:", e)




Data successfully dumped to table 'main' in PostgreSQL!


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Example connection string (modify based on your database)
DB_HOST = "rgx.ddns.net"  # e.g., "localhost"
DB_PORT = "5432"  # Default PostgreSQL port
DB_NAME = "Capstone"
DB_USER = "postgres"
DB_PASSWORD = "Google#13"
TABLE_NAME = "main"
CHUNK_SIZE = 600  # Adjust as needed

# Create a SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# SQL query to fetch user_id and text
query = "SELECT user_id, text FROM main limit 10;"

# Read into DataFrame
df = pd.read_sql(query, engine)

# Display DataFrame
df.head()


,user_id,text
0,1.003082e+20,None
1,1.003082e+20,None
2,1.118020e+20,None
3,1.003082e+20,None
4,1.176197e+20,None


In [3]:
from nltk.tokenize import word_tokenize
text = "Hello, this is a test."
print(word_tokenize(text))


['Hello', ',', 'this', 'is', 'a', 'test', '.']


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word.lower() not in set(stopwords.words('english'))]

    return ' '.join(tokens)

df['clean_text'] = df['text'].apply(clean_text)



In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
def get_sentiment(text):
    sentiment_score = sia.polarity_scores(text)['compound']
    
    # Classify sentiment
    if sentiment_score >= 0.05:
        sentiment = "Positive"
    elif sentiment_score <= -0.05:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"
    
    return sentiment_score, sentiment

# Apply sentiment analysis
df[['sentiment_score', 'sentiment_category']] = df['clean_text'].apply(lambda x: pd.Series(get_sentiment(x)))


In [ ]:
pip install ace_tools

In [ ]:
# Display DataFrame
import ace_tools as tools
tools.display_dataframe_to_user(name="Sentiment Analysis Data", dataframe=df)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Example connection string (modify based on your database)
DB_HOST = "localhost"  # e.g., "localhost"
DB_PORT = "5432"  # Default PostgreSQL port
DB_NAME = "Capstone"
DB_USER = "postgres"
DB_PASSWORD = "Google#13"
TABLE_NAME = "main"
CHUNK_SIZE = 600  # Adjust as needed

# Create a SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# SQL query to fetch user_id and text
query = "SELECT gmap_id, latitude, longitude FROM main"

# Read into DataFrame
df_loc = pd.read_sql(query, engine)

# Display DataFrame
df_loc.head()


In [ ]:
pip install geopy


In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import pandas as pd

# Initialize geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

def get_zipcode(lat, lon):
    """
    Fetches ZIP code from latitude and longitude using Nominatim.
    """
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if location and 'postcode' in location.raw['address']:
            return location.raw['address']['postcode']
        else:
            return None  # No ZIP code found
    except GeocoderTimedOut:
        return "Timeout"

# Example latitude and longitude values


# Apply function to get ZIP codes
df_loc['zip_code'] = df_loc.apply(lambda row: get_zipcode(row['latitude'], row['longitude']), axis=1)


